In [1]:
from os import listdir

import time
import json
import copy

import torch

from torchvision.datasets import mnist, FashionMNIST, CIFAR10, CIFAR100
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torch.optim import SGD, Adam
from torch.nn import Module
from torch import nn
from torch.nn import CrossEntropyLoss
from torchvision.models.resnet import Bottleneck, ResNet
from torchvision import datasets, models, transforms

from wilds import get_dataset
from wilds.common.data_loaders import get_train_loader
import torchvision.transforms as transforms

import numpy as np
import pandas as pd

from openood.evaluators import metrics

In [2]:
print(torch.version.cuda)

10.2


In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [4]:
!pwd

/home/rdr2143/oodn-final-project/OpenOOD-nndl


### Supported Activation Functions

For activation functions, we are considering ReLU, Softplus, Swish. *Note that we may conduct experiments for a subset based on the compute resources available*

In [5]:
def get_activation_fn(activation):
    if activation == 'relu':
        return nn.ReLU()
    elif activation == 'softplus':
        return nn.Softplus()
    elif activation == 'swish':
        return nn.Swish()
    return None

### LeNet

In [6]:
class LeNet(nn.Module):
    def __init__(self, num_classes, num_channel=3, activation='relu'):
        super(LeNet, self).__init__()
        self.num_classes = num_classes
        self.feature_size = 84
        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels=num_channel,
                      out_channels=6,
                      kernel_size=5,
                      stride=1,
                      padding=2), get_activation_fn(activation), nn.MaxPool2d(kernel_size=2))

        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
             get_activation_fn(activation), nn.MaxPool2d(kernel_size=2))

        self.block3 = nn.Sequential(
            nn.Conv2d(in_channels=16,
                      out_channels=120,
                      kernel_size=5,
                      stride=1), get_activation_fn(activation))

        self.classifier1 = nn.Linear(in_features=120, out_features=84)
        self.relu = get_activation_fn(activation)
        self.fc = nn.Linear(in_features=84, out_features=num_classes)

    def get_fc(self):
        fc = self.fc
        return fc.weight.cpu().detach().numpy(), fc.bias.cpu().detach().numpy()

    def forward(self, x, return_feature=False, return_feature_list=False):
        feature1 = self.block1(x)
        feature2 = self.block2(feature1)
        feature3 = self.block3(feature2)
        feature3 = feature3.view(feature3.shape[0], -1)
        feature = self.relu(self.classifier1(feature3))
        logits_cls = self.fc(feature)
        feature_list = [feature1, feature2, feature3, feature]
        if return_feature:
            return logits_cls, feature
        elif return_feature_list:
            return logits_cls, feature_list
        else:
            return logits_cls

    def forward_threshold(self, x, threshold):
        feature1 = self.block1(x)
        feature2 = self.block2(feature1)
        feature3 = self.block3(feature2)
        feature3 = feature3.view(feature3.shape[0], -1)
        feature = self.relu(self.classifier1(feature3))
        feature = feature.clip(max=threshold)
        logits_cls = self.fc(feature)

        return logits_cls

### ResNet50

In [7]:
def set_parameter_requires_grad(model):
    for name,param in model.named_parameters():
        if not (name.startswith('layer4') or name.startswith('fc')):
            param.requires_grad = False

def get_resnet_model(activation_function_type, n_classes, use_pretrained=True):
    resnet_model = models.resnet50(pretrained=use_pretrained)
    
    # if we use pretrained, then freeze the corresponding layers
    if use_pretrained:
        set_parameter_requires_grad(resnet_model, feature_extract)

    set_activation_function(resnet_model,activation_function_type)
    num_ftrs = resnet_model.fc.in_features
    resnet_model.fc = nn.Linear(num_ftrs, n_classes)
    resnet_model.to(device)
    return resnet_model

def set_activation_function(resnet_model, activation_function_type):
    resnet_model.relu = get_activation_fn(activation_function_type)
    resnet_model.layer1[0].relu = get_activation_fn(activation_function_type)
    resnet_model.layer1[1].relu = get_activation_fn(activation_function_type)
    resnet_model.layer1[2].relu = get_activation_fn(activation_function_type)

    resnet_model.layer2[0].relu = get_activation_fn(activation_function_type)
    resnet_model.layer2[1].relu = get_activation_fn(activation_function_type)
    resnet_model.layer2[2].relu = get_activation_fn(activation_function_type)
    resnet_model.layer2[3].relu = get_activation_fn(activation_function_type)

    resnet_model.layer3[0].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[1].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[2].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[3].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[4].relu = get_activation_fn(activation_function_type)
    resnet_model.layer3[5].relu = get_activation_fn(activation_function_type)


    resnet_model.layer4[0].relu = get_activation_fn(activation_function_type)
    resnet_model.layer4[1].relu = get_activation_fn(activation_function_type)
    resnet_model.layer4[2].relu = get_activation_fn(activation_function_type)

    return resnet_model

In [8]:
def get_model(config):
    activation_function_type = config["activation_function_type"]
    network_type = config["network"]
    n_classes = config["n_classes"]

    if network_type == "lenet":
        model =  LeNet(num_classes=n_classes, num_channel=1, activation=activation_function_type)
    elif network_type == "resnet50":
        model = get_resnet_model(activation_function_type, n_classes, config['pretrained'])
    else:
        raise Exception("Currently we only support lenet or resnet50")

    return model

### Supported Post-Hoc OODN Processors

In [9]:
class OODPostprocessor():
    
    def inference(self, net: nn.Module, data_loader: DataLoader):
        pred_list, conf_list, label_list = [], [], []
        for idx, loaded_data in enumerate(data_loader):
            data, label = loaded_data[0], loaded_data[1]
            if idx % 50 == 0:
                print(f'Performing inference on batch: {idx}')
            pred, conf = self.postprocess(net, data.to(device))
            for idx in range(len(data)):
                pred_list.append(pred[idx].tolist())
                conf_list.append(conf[idx].tolist())
                label_list.append(label[idx].tolist())

        # convert values into numpy array
        pred_list = np.array(pred_list, dtype=int)
        conf_list = np.array(conf_list)
        label_list = np.array(label_list, dtype=int)

        return pred_list, conf_list, label_list

In [10]:
class ODINPostprocessor(OODPostprocessor):
    def __init__(self, temperature, noise):
        super(OODPostprocessor)
        self.temperature = temperature
        self.noise = noise
        
    def postprocess(self, net: nn.Module, data):
        net.eval()
        data.requires_grad = True
        output = net(data)

        # Calculating the perturbation we need to add, that is,
        # the sign of gradient of cross entropy loss w.r.t. input
        criterion = nn.CrossEntropyLoss()

        labels = output.detach().argmax(axis=1)

        # Using temperature scaling
        output = output / self.temperature

        loss = criterion(output, labels)
        loss.backward()

        # Normalizing the gradient to binary in {0, 1}
        gradient = torch.ge(data.grad.detach(), 0)
        gradient = (gradient.float() - 0.5) * 2

        # Scaling values taken from original code       
        gradient[:, 0] = (gradient[:, 0]) / (63.0 / 255.0)
        if gradient.shape[1] == 3:
            gradient[:, 1] = (gradient[:, 1]) / (62.1 / 255.0)
            gradient[:, 2] = (gradient[:, 2]) / (66.7 / 255.0)

        # Adding small perturbations to images
        tempInputs = torch.add(data.detach(), gradient, alpha=-self.noise)
        output = net(tempInputs)
        output = output / self.temperature

        # Calculating the confidence after adding perturbations
        nnOutput = output.detach()
        nnOutput = nnOutput - nnOutput.max(dim=1, keepdims=True).values
        nnOutput = nnOutput.exp() / nnOutput.exp().sum(dim=1, keepdims=True)

        conf, pred = nnOutput.max(dim=1)

        return pred, conf

In [11]:
class MCDPostprocessor(OODPostprocessor):
    def __init__(self, samples: int = 30):
        super(OODPostprocessor)
        self.samples = samples  #: number :math:`N` of samples

    def postprocess(self, model: torch.nn.Module, x: torch.Tensor) -> torch.Tensor:
        mode_switch = False
        if not model.training:
            mode_switch = True

            model.train()

            for mod in model.modules():
                # reset batch norm layers.
                # TODO: are there other layers?
                if isinstance(mod, (nn.BatchNorm1d, nn.BatchNorm2d, nn.BatchNorm3d)):
                    mod.train(False)

        results = None
        with torch.no_grad():
            for i in range(self.samples):
                output = model(x).softmax(dim=1)
                if results is None:
                    results = torch.zeros(size=output.shape).to(device)
                results += output
        results /= self.samples

        if mode_switch:
            model.eval()
        
        conf, pred = results.max(dim=1)

        return pred, conf

In [12]:
def get_postprocessor(postprocessor_type="odin"):
    if postprocessor_type == "odin":
        postprocessor = ODINPostprocessor(1000, 0.0014)
    elif postprocessor_type == "mcd":
        postprocessor = MCDPostprocessor(30)
    return postprocessor

### Supported Out of Distribution Detection Metrics

What metrics do we specifically care about here?

**FPR@95** measures the false positive rate (FPR) when the true positive rate (TPR) is
equal to 95%. Lower scores indicate better performance.

**AUROC** measures the area under the
Receiver Operating Characteristic (ROC) curve, which displays the relationship between TPR and
FPR. The area under the ROC curve can be interpreted as the probability that a positive ID example
will have a higher detection score than a negative OOD example.

**AUPR** measures the area under
the Precision-Recall (PR) curve. The PR curve is created by plotting precision versus recall. Similar
to AUROC, we consider ID samples as positive, so that the score corresponds to the AUPR-In metric
in some works

In [13]:
def calculate_oodn_metrics(model, postprocessor_type, id_test_loader, ood_test_loader, ood_name):
    postprocessor = get_postprocessor(postprocessor_type)
    id_pred, id_conf, id_gt = postprocessor.inference(
                model, id_test_loader)

    ood_pred, ood_conf, ood_gt = postprocessor.inference(
        model, ood_test_loader)

    ood_gt = -1 * np.ones_like(ood_gt)  # hard set to -1 as ood
    pred = np.concatenate([id_pred, ood_pred])
    conf = np.concatenate([id_conf, ood_conf])
    label = np.concatenate([id_gt, ood_gt])
    ood_metrics = metrics.compute_all_metrics(conf, label, pred)

    return print_and_get_formatted_metrics(ood_metrics, ood_name)

def print_and_get_formatted_metrics(metrics, dataset_name):
    [fpr, auroc, aupr_in, aupr_out,
     ccr_4, ccr_3, ccr_2, ccr_1, accuracy] \
     = metrics

    write_content = {
        'dataset': dataset_name,
        'FPR@95': '{:.2f}'.format(100 * fpr),
        'AUROC': '{:.2f}'.format(100 * auroc),
        'AUPR_IN': '{:.2f}'.format(100 * aupr_in),
        'AUPR_OUT': '{:.2f}'.format(100 * aupr_out),
        'CCR_4': '{:.2f}'.format(100 * ccr_4),
        'CCR_3': '{:.2f}'.format(100 * ccr_3),
        'CCR_2': '{:.2f}'.format(100 * ccr_2),
        'CCR_1': '{:.2f}'.format(100 * ccr_1),
        'ACC': '{:.2f}'.format(100 * accuracy)
    }

    fieldnames = list(write_content.keys())

    # print ood metric results
    print('FPR@95: {:.2f}, AUROC: {:.2f}'.format(100 * fpr, 100 * auroc),
          end=' ',
          flush=True)
    print('AUPR_IN: {:.2f}, AUPR_OUT: {:.2f}'.format(
        100 * aupr_in, 100 * aupr_out),
          flush=True)
    print('CCR: {:.2f}, {:.2f}, {:.2f}, {:.2f},'.format(
        ccr_4 * 100, ccr_3 * 100, ccr_2 * 100, ccr_1 * 100),
          end=' ',
          flush=True)
    print('ACC: {:.2f}'.format(accuracy * 100), flush=True)
    print(u'\u2500' * 70, flush=True)
    return write_content

def load_results_into_df(dir_path):
    res_files = [dir_path+each for each in listdir(dir_path)]
    all_results = []
    columns = ['optimizer_type', 'activation_function_type', 'postprocessor_type', 'trial', 'AUROC']
    for fp in res_files:
        f = open(fp)
        data = json.load(f)
        for trial, results in data.items():
            all_results.append([
                    results['optimizer_type'],
                    results['activation_function_type'],
                    results['postprocessor_type'],
                    trial,
                    float(results['AUROC'])
                ])
    df = pd.DataFrame(all_results, columns=columns)
    return df

In [25]:
def get_optimizer(model, config):
    params = model.parameters()
    lr = config['lr']
    momentum = config['momentum']
    weight_decay = config['weight_decay']
    optimizer_type = config['optimizer_type']

    print(f'Getting optimizer for type: {optimizer_type}...')
    if optimizer_type == 'SGD':
        return SGD(params,
              lr=lr,
              momentum=momentum,
              weight_decay=weight_decay)
    elif optimizer_type == 'Adam':
        return Adam(params,
                    lr=lr)
    else:
        raise Exception("Invalid optimizer_type provided, only SGD and Adam are supported currently")

def get_wilds_loader(dataset, split, batch_size):
    d = dataset.get_subset(
        split,
        # frac=0.1,
        transform=transforms.Compose(
            [transforms.Resize((448, 448)), transforms.ToTensor()]
        ),
    )
    # Prepare the standard data loader
    return get_train_loader("standard", d, batch_size=batch_size, num_workers=4)

def get_data_loaders(config):
    data_loaders = {}
    dataset_name = config["dataset_name"]
    dataset_type = config["dataset_type"]
    batch_size = config['batch_size']

    wilds_id_test_split = "id_val" if dataset_name == "camelyon17" else "id_test"
    if dataset_type == "wilds":
        # wilds dataset
        dataset = get_dataset(dataset=dataset_name, download=True)
        data_loaders["train"] = get_wilds_loader(dataset, "train", batch_size)
        data_loaders["ood_test"] = get_wilds_loader(dataset, "test", batch_size)
        data_loaders["id_test"] = get_wilds_loader(dataset, wilds_id_test_split, batch_size)
        return
    elif dataset_name == "cifar":
        transform_train = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])

        transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])
        train_dataset = CIFAR10(root='data', download=True, train=True, transform=transform_train)
        test_dataset = CIFAR10(root='data', download=True, train=False, transform=transform_test)
        ood_test_dataset = CIFAR100(root='data', download=True, train=False, transform=transform_test)
        
    elif dataset_name == "mnist":
        # mnist dataset
        train_dataset = mnist.MNIST(root='data', download=True, train=True, transform=ToTensor())
        test_dataset = mnist.MNIST(root='data', download=True, train=False, transform=ToTensor())
        ood_test_dataset = mnist.FashionMNIST(root='data', download=True,train=False,transform=ToTensor())

    data_loaders["train"] = DataLoader(train_dataset, batch_size=batch_size, num_workers=4)
    data_loaders["id_test"] = DataLoader(test_dataset, batch_size=batch_size, num_workers=4)
    data_loaders["ood_test"] = DataLoader(ood_test_dataset, batch_size=batch_size, num_workers=4)

    return data_loaders

In [26]:
def train_resnet_model_given_opti_activation_fn(config):
    # get the train loader
    train_loader = config["data_loaders"]["train"]

    # get the resnet model with the replaced activation functions
    model = get_model(config)
    model.to(device)

    # get the optimizer
    sgd = get_optimizer(model, config)

    loss_fn = CrossEntropyLoss()
    for current_epoch in range(config['epochs']):
        tic=time.time()
        per_batch_time = time.time()
        model.train()
        print('Training epoch: {}'.format(current_epoch))
        for idx, (loader_data) in enumerate(train_loader):
            train_x, train_label = loader_data[0].to(device), loader_data[1].to(device)
            sgd.zero_grad()
            predict_y = model(train_x.float())
            loss = loss_fn(predict_y, train_label.long())
            if idx % 100 == 0:
                print('idx: {}, loss: {} time take: {}'.format(idx, loss.sum().item(), time.time() - per_batch_time))
                per_batch_time = time.time()
            loss.backward()
            sgd.step()
        print(f"epoch {current_epoch} time taken: {time.time()-tic}s")
    torch.save(model, config['model_name'])

    return model

def run_full_oodn_pipeline(config):
    metrics = {}
    for i in range(config["trials"]):
        model_name = f"models/{config['dataset_name']}_{config['network']}_{config['postprocessor_type']}_{config['activation_function_type']}_{config['optimizer_type']}_{i}.pkl"
        print(f'Running model: {model_name}...')
        config['model_name'] = model_name
        # train model
        model = train_resnet_model_given_opti_activation_fn(config)
        # calculate oodn metrics
        metrics[i] = calculate_oodn_metrics(model,
                               config['postprocessor_type'],
                               config["data_loaders"]["id_test"],
                               config["data_loaders"]["ood_test"],
                               config["dataset_name"])
        metrics[i]['optimizer_type'] = config['optimizer_type']
        metrics[i]['activation_function_type'] = config['activation_function_type']
        metrics[i]['postprocessor_type'] = config['postprocessor_type']

    experiment_name = f"{config['results_dir']}/{config['dataset_name']}_{config['network']}_{config['postprocessor_type']}_{config['activation_function_type']}_{config['optimizer_type']}.json"
    with open(experiment_name, 'w') as fp:
        json.dump(metrics, fp)
    return metrics

### Study 2: Resnet, CIFAR-10 (ID), CIFAR-100 (OOD)

#### Study 2(a): Adam + ReLU + Odin

In [27]:
config_cifar_adam_relu_odin = {
    "batch_size": 128,
    "n_classes": 10,
    "dataset_name": "cifar",
    "epochs": 100,
    "version": time.time(),
    "lr": 0.01,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "Adam",
    "activation_function_type": "relu",
    "network": "resnet50",
    "postprocessor_type": "odin",
    "trials": 3,
    "dataset_type": "cifar",
    "results_dir": "cifar10-study",
    "pretrained": False
} 
config_cifar_adam_relu_odin["data_loaders"] = get_data_loaders(config_cifar_adam_relu_odin)
run_full_oodn_pipeline(config_cifar_adam_relu_odin)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Running model: models/cifar_resnet50_odin_relu_Adam_0.pkl...
Getting optimizer for type: Adam...
Training epoch: 0
idx: 0, loss: 2.8178350925445557 time take: 0.2426450252532959
idx: 100, loss: 3.1174871921539307 time take: 6.610335350036621
idx: 200, loss: 2.9341554641723633 time take: 6.59183406829834
idx: 300, loss: 1.8614654541015625 time take: 6.668043613433838
epoch 0 time taken: 26.097416400909424s
Training epoch: 1
idx: 0, loss: 1.8956336975097656 time take: 0.2481977939605713
idx: 100, loss: 1.6939016580581665 time take: 6.58957839012146
idx: 200, loss: 1.5685464143753052 time take: 6.630019426345825
idx: 300, loss: 1.6713684797286987 time take: 6.604220151901245
epoch 1 time taken: 26.034528970718384s
Training epoch: 2
idx: 0, loss: 1.499733567237854 time take: 0.25389885902404785
idx: 100, loss: 1.3913094997406006 time take: 6.598061800003052
idx: 200, loss: 1.46

idx: 300, loss: 0.5411957502365112 time take: 6.709514379501343
epoch 25 time taken: 26.26641321182251s
Training epoch: 26
idx: 0, loss: 0.6170827746391296 time take: 0.2566368579864502
idx: 100, loss: 0.5026325583457947 time take: 6.604597091674805
idx: 200, loss: 0.6255069375038147 time take: 6.596994161605835
idx: 300, loss: 0.6075899004936218 time take: 6.639557123184204
epoch 26 time taken: 26.081337213516235s
Training epoch: 27
idx: 0, loss: 0.403511643409729 time take: 0.26786375045776367
idx: 100, loss: 0.38416874408721924 time take: 6.615552186965942
idx: 200, loss: 0.5412840247154236 time take: 6.61034631729126
idx: 300, loss: 0.4574718773365021 time take: 6.618563652038574
epoch 27 time taken: 26.1037859916687s
Training epoch: 28
idx: 0, loss: 0.3946287930011749 time take: 0.26188039779663086
idx: 100, loss: 0.5084614753723145 time take: 6.697983503341675
idx: 200, loss: 0.6018609404563904 time take: 6.6684346199035645
idx: 300, loss: 0.3823794722557068 time take: 6.69977903

{0: {'dataset': 'cifar',
  'FPR@95': '73.75',
  'AUROC': '81.36',
  'AUPR_IN': '83.13',
  'AUPR_OUT': '78.00',
  'CCR_4': '3.19',
  'CCR_3': '6.01',
  'CCR_2': '22.05',
  'CCR_1': '53.57',
  'ACC': '83.93',
  'optimizer_type': 'Adam',
  'activation_function_type': 'relu',
  'postprocessor_type': 'odin'}}

#### Study 2 (b.) Adam + Softplus + odin

In [28]:
config_cifar_adam_softplus_odin = {
    "batch_size": 128,
    "n_classes": 10,
    "dataset_name": "cifar",
    "epochs": 100,
    "version": time.time(),
    "lr": 0.01,
    "momentum": 0.9,
    "weight_decay": 0.0005,
    "optimizer_type": "Adam",
    "activation_function_type": "softplus",
    "network": "resnet50",
    "postprocessor_type": "odin",
    "trials": 3,
    "dataset_type": "cifar",
    "results_dir": "cifar10-study",
    "pretrained": False
}
config_cifar_adam_softplus_odin["data_loaders"] = get_data_loaders(config_cifar_adam_softplus_odin)
run_full_oodn_pipeline(config_cifar_adam_softplus_odin)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Running model: models/cifar_resnet50_odin_softplus_Adam_0.pkl...
Getting optimizer for type: Adam...
Training epoch: 0
idx: 0, loss: 2.5709333419799805 time take: 0.24824285507202148
idx: 100, loss: 3.132185459136963 time take: 6.721421241760254
idx: 200, loss: 3.2185091972351074 time take: 6.609935283660889
idx: 300, loss: 2.3499503135681152 time take: 6.611340045928955
epoch 0 time taken: 26.187455892562866s
Training epoch: 1
idx: 0, loss: 3.0913891792297363 time take: 0.267345666885376
idx: 100, loss: 2.5008647441864014 time take: 6.636933088302612
idx: 200, loss: 2.9766957759857178 time take: 6.615807056427002
idx: 300, loss: 1.9372632503509521 time take: 6.628815650939941
epoch 1 time taken: 26.139310121536255s
Training epoch: 2
idx: 0, loss: 2.0130720138549805 time take: 0.25510263442993164
idx: 100, loss: 1.980995774269104 time take: 6.6487414836883545
idx: 200, loss

idx: 200, loss: 0.5319328308105469 time take: 6.623592853546143
idx: 300, loss: 0.48465731739997864 time take: 6.611387491226196
epoch 25 time taken: 26.133150577545166s
Training epoch: 26
idx: 0, loss: 0.4033209979534149 time take: 0.23418903350830078
idx: 100, loss: 0.5307964086532593 time take: 6.6245338916778564
idx: 200, loss: 0.6812744140625 time take: 6.607555627822876
idx: 300, loss: 0.4704570174217224 time take: 6.628464460372925
epoch 26 time taken: 26.16669535636902s
Training epoch: 27
idx: 0, loss: 0.44897329807281494 time take: 0.2201521396636963
idx: 100, loss: 0.39711782336235046 time take: 6.631270170211792
idx: 200, loss: 0.48680832982063293 time take: 6.607851505279541
idx: 300, loss: 0.4265068471431732 time take: 6.612071752548218
epoch 27 time taken: 26.070226430892944s
Training epoch: 28
idx: 0, loss: 0.4318372309207916 time take: 0.25647926330566406
idx: 100, loss: 0.4094410836696625 time take: 6.631103754043579
idx: 200, loss: 0.552060067653656 time take: 6.61326

{0: {'dataset': 'cifar',
  'FPR@95': '76.86',
  'AUROC': '80.02',
  'AUPR_IN': '81.92',
  'AUPR_OUT': '76.03',
  'CCR_4': '0.00',
  'CCR_3': '1.65',
  'CCR_2': '22.50',
  'CCR_1': '50.67',
  'ACC': '82.72',
  'optimizer_type': 'Adam',
  'activation_function_type': 'softplus',
  'postprocessor_type': 'odin'}}